In [1]:
import os
os.environ["THEANO_FLAGS"] = 'cuda.root=/usr/local/cuda,floatX=float32,device=gpu1,force_device=False,lib.cnmem=.75'

import theano
print(theano.config.device)

import mhcflurry, seaborn, numpy, pandas, pickle, sklearn, collections, scipy, time
import mhcflurry.data
import mhcflurry.imputation
import fancyimpute, locale

import sklearn.metrics
import sklearn.cross_validation

Using gpu device 1: GeForce GTX TITAN X (CNMeM is enabled with initial size: 75.0% of memory, cuDNN 5004)
Using Theano backend.


gpu1


In [2]:
max_ic50 = 50000
min_peptides_to_consider_allele = 10
data_dir = "../data/"

In [3]:
all_train_data = mhcflurry.data.load_allele_datasets(data_dir + "bdata.2009.mhci.public.1.txt")
#all_validation_data = mhcflurry.data.load_allele_datasets(data_dir + "bdata.2013.mhci.public.blind.1.txt")


In [4]:
imputed_train_data = mhcflurry.imputation.create_imputed_datasets(
    all_train_data,
    fancyimpute.MICE(),
    min_observations_per_peptide=2,
    min_observations_per_allele=10
)


Dropping 12235 peptides with <2 observations
Dropping 9 alleles with <10 observations: ['ELA-A1', 'HLA-B2701', 'HLA-B3508', 'HLA-B44', 'HLA-E0101', 'Mamu-B04', 'Patr-A0602', 'Patr-B0901', 'Patr-B1701']
[MICE] Completing matrix with shape (19304, 97)
[MICE] Starting imputation round 1/110, elapsed time 0.023
[MICE] Starting imputation round 2/110, elapsed time 1.659
[MICE] Starting imputation round 3/110, elapsed time 3.079
[MICE] Starting imputation round 4/110, elapsed time 4.573
[MICE] Starting imputation round 5/110, elapsed time 5.993
[MICE] Starting imputation round 6/110, elapsed time 7.489
[MICE] Starting imputation round 7/110, elapsed time 9.023
[MICE] Starting imputation round 8/110, elapsed time 10.403
[MICE] Starting imputation round 9/110, elapsed time 11.871
[MICE] Starting imputation round 10/110, elapsed time 13.326
[MICE] Starting imputation round 11/110, elapsed time 14.847
[MICE] Starting imputation round 12/110, elapsed time 16.409
[MICE] Starting imputation round 1

In [5]:
validation_df = pandas.read_csv("../data/combined_test_BLIND_dataset_from_kim2013.csv")
validation_df

,allele,peptide,length,meas,netmhc,netmhcpan,smmpmbec_cpp
0,H-2-DB,AAACNVATA,9,657.657837,154.881662,711.213514,438.530698
1,H-2-DB,AAFEFVYV,8,30831.879502,6456.542290,785.235635,10351.421667
2,H-2-DB,AAFVNDYSL,9,77.446180,17.458222,7.516229,28.054336
3,H-2-DB,AAIANQAAV,9,1.999862,9.638290,9.749896,25.703958
4,H-2-DB,AAIANQAVV,9,1.517050,8.550667,8.336812,28.773984
5,H-2-DB,AAIENYVRF,9,37.844258,252.348077,114.815362,187.068214
6,H-2-DB,AAINFITTM,9,3.155005,199.986187,389.045145,200.909281
7,H-2-DB,AAIPAPPPI,9,3243.396173,1059.253725,493.173804,295.120923
8,H-2-DB,AAKLNRPPL,9,654.636174,66.374307,77.268059,38.459178
9,H-2-DB,AALDMVDAL,9,229.614865,547.015963,597.035287,225.423921


In [6]:
alleles = sorted((
    allele for allele in all_train_data
    if len(set(all_train_data[allele].original_peptides)) >= min_peptides_to_consider_allele),
                 key=lambda allele: -1 * len(set(all_train_data[allele].original_peptides)))
alleles

print(len(alleles))
alleles

91


['HLA-A0201',
 'HLA-A0301',
 'HLA-A0203',
 'HLA-A1101',
 'HLA-A0206',
 'HLA-A3101',
 'HLA-A6802',
 'HLA-A0202',
 'HLA-A0101',
 'HLA-B0702',
 'H-2-KB',
 'H-2-DB',
 'HLA-B1501',
 'HLA-A6801',
 'HLA-A3301',
 'HLA-B2705',
 'HLA-A2601',
 'HLA-B4001',
 'HLA-B5801',
 'HLA-A2402',
 'HLA-A2902',
 'HLA-B3501',
 'HLA-B0801',
 'Mamu-A01',
 'HLA-A6901',
 'HLA-B1801',
 'HLA-A3001',
 'HLA-A2301',
 'HLA-B5701',
 'HLA-B5101',
 'HLA-B4402',
 'Mamu-B17',
 'HLA-A3002',
 'HLA-B4601',
 'Mamu-A11',
 'HLA-A0219',
 'HLA-A2403',
 'HLA-A0212',
 'Mamu-B03',
 'Mamu-B08',
 'HLA-A0211',
 'HLA-B5401',
 'HLA-B5301',
 'Mamu-A02',
 'HLA-B4403',
 'HLA-A0216',
 'HLA-B4501',
 'HLA-B3901',
 'HLA-B4002',
 'HLA-B4801',
 'HLA-B1517',
 'Mamu-B01',
 'HLA-A8001',
 'Patr-B0101',
 'HLA-A3201',
 'Patr-A0901',
 'Patr-A0701',
 'HLA-A2501',
 'HLA-B0802',
 'Mamu-A2201',
 'H-2-KD',
 'HLA-B2703',
 'HLA-B1503',
 'Patr-A0101',
 'H-2-KK',
 'HLA-B1509',
 'Patr-B2401',
 'Patr-A0301',
 'H-2-DD',
 'H-2-LD',
 'Patr-A0401',
 'HLA-B0803',
 'HLA-A26

In [7]:
dropout_probabilities = [0.0, 0.5]
embedding_output_dims_and_layer_sizes_list = [(32, [64]), (8, [4])]
activations = ["tanh"]

models_params_list = []

for impute in [False, True]:
    for dropout_probability in dropout_probabilities:
        for (embedding_output_dim, layer_sizes) in embedding_output_dims_and_layer_sizes_list:
            for activation in activations:
                models_params_list.append(dict(
                    impute=impute,
                    dropout_probability=dropout_probability,  
                    embedding_output_dim=embedding_output_dim,
                    layer_sizes=layer_sizes,
                    activation=activation))

print("%d models" % len(models_params_list))
models_params_explored = set.union(*[set(x) for x in models_params_list])
models_params_explored


8 models


{'activation',
 'dropout_probability',
 'embedding_output_dim',
 'impute',
 'layer_sizes'}

In [8]:
def log_to_ic50(log_value):
        """
        Convert neural network output to IC50 values between 0.0 and
        self.max_ic50 (typically 5000, 20000 or 50000)
        """
        return max_ic50 ** (1.0 - log_value)

def make_scores(ic50_y, ic50_y_pred, sample_weight=None, threshold_nm=500):     
    y_pred = mhcflurry.common.ic50_to_regression_target(ic50_y_pred, max_ic50)
    try:
        auc = sklearn.metrics.roc_auc_score(ic50_y <= threshold_nm, y_pred, sample_weight=sample_weight)
    except ValueError:
        auc = numpy.nan
    try:
        f1 = sklearn.metrics.f1_score(ic50_y <= threshold_nm, ic50_y_pred <= threshold_nm, sample_weight=sample_weight)
    except ValueError:
        f1 = numpy.nan
    try:
        tau = scipy.stats.kendalltau(ic50_y_pred, ic50_y)[0]
    except ValueError:
        tau = numpy.nan
    
    return dict(
        auc=auc,
        f1=f1,
        tau=tau,
    )    

def mean_with_std(grouped_column, decimals=3):
    pattern = "%%0.%df" % decimals
    return pandas.Series([
        (pattern + " +/ " + pattern) % (m, s) if not pandas.isnull(s) else pattern % m
        for (m, s) in zip(grouped_column.mean(), grouped_column.std())
    ], index = grouped_column.mean().index)

def allele_data_to_df(data):
    d = data._asdict()
    d["X_index"] = [x for x in d["X_index"]]
    d["X_binary"] = [x for x in d["X_binary"]]
    df = pandas.DataFrame(d).set_index('peptides')
    return df

def make_2d_array(thing):
    return numpy.array([list(x) for x in thing])

def df_to_allele_data(df):
    d = dict((col, df[col].values) for col in df)
    d["X_index"] = make_2d_array(d["X_index"])
    (d["max_ic50"],) = list(df.max_ic50.unique())
    return mhcflurry.data.AlleleData(peptides = df.index.values, **d)


In [9]:
a=all_train_data["Mamu-A02"]

In [10]:
models_and_scores = {}
validation_df_with_mhcflurry = validation_df.copy()


6027

In [23]:
# train and test models, adding columns to validation_df_with_mhcflurry
def make_and_fit_model(allele, original_params):
    params = dict(original_params)
    impute = params["impute"]
    del params["impute"]
    model = mhcflurry.Class1BindingPredictor.from_hyperparameters(max_ic50=max_ic50, **params)
    print("Fitting model for allele %s (%d + %d): %s" % (
            allele, len(all_train_data[allele].Y), len(imputed_train_data[allele].Y), str(original_params)))
    t = -time.time()
    model.fit(all_train_data[allele].X_index,
              all_train_data[allele].Y,
              sample_weights=all_train_data[allele].weights,
              X_pretrain=imputed_train_data[allele].X_index if impute else None,
              Y_pretrain=imputed_train_data[allele].Y if impute else None,
              sample_weights_pretrain=imputed_train_data[allele].weights if impute else None,
              verbose=False,
              batch_size=128,
              n_training_epochs=250)
    t += time.time()
    print("Trained in %d sec" % t)
    return model

for (i, allele) in enumerate(alleles[:2]):
    if allele not in validation_df_with_mhcflurry.allele.unique():
        print("Skipping allele %s: not in test set" % allele)
        continue
    values_for_allele = []
    for (j, params) in enumerate(models_params_list):
        print("Allele %d model %d" % (i, j))
        model = make_and_fit_model(allele, params)
        predictions = model.predict_peptides_ic50(
            list(validation_df_with_mhcflurry.ix[validation_df_with_mhcflurry.allele == allele].peptide))
        print("test set size: %d" % len(predictions))
        validation_df_with_mhcflurry.loc[(validation_df_with_mhcflurry.allele == allele),
                                         ("mhcflurry %d" % j)] = predictions
        scores = make_scores(validation_df_with_mhcflurry.ix[validation_df.allele == allele].meas,
                            predictions)
        print(scores)
        values_for_allele.append((params, model, scores))
        
    models_and_scores[allele] = values_for_allele
        


Allele 0 model 0
Fitting model for allele HLA-A0201 (32876 + 80965): {'activation': 'tanh', 'impute': False, 'embedding_output_dim': 32, 'dropout_probability': 0.0, 'layer_sizes': [64]}
Trained in 38 sec
test set size: 2126
{'tau': 0.55498762313021643, 'auc': 0.89912294760836253, 'f1': 0.83782589482987169}
Allele 0 model 1
Fitting model for allele HLA-A0201 (32876 + 80965): {'activation': 'tanh', 'impute': False, 'embedding_output_dim': 8, 'dropout_probability': 0.0, 'layer_sizes': [4]}
Trained in 34 sec
test set size: 2126
{'tau': 0.6233138026840126, 'auc': 0.92685294012893105, 'f1': 0.87067075843928099}
Allele 0 model 2
Fitting model for allele HLA-A0201 (32876 + 80965): {'activation': 'tanh', 'impute': False, 'embedding_output_dim': 32, 'dropout_probability': 0.5, 'layer_sizes': [64]}
Trained in 44 sec
test set size: 2126
{'tau': 0.6225574009974717, 'auc': 0.92571321722406241, 'f1': 0.88116688116688113}
Allele 0 model 3
Fitting model for allele HLA-A0201 (32876 + 80965): {'activatio

In [28]:
validation_df_with_mhcflurry_results = validation_df_with_mhcflurry.ix[validation_df_with_mhcflurry.allele.isin(models_and_scores)]
validation_df_with_mhcflurry_results

,allele,peptide,length,meas,netmhc,netmhcpan,smmpmbec_cpp,mhcflurry 0,mhcflurry 1,mhcflurry 2,mhcflurry 3,mhcflurry 4,mhcflurry 5,mhcflurry 6,mhcflurry 7
2047,HLA-A0201,AAAFVNQHL,9,5011.872336,10889.300933,16069.412530,6.576578e+03,3887.631590,17564.810701,8310.155400,5069.018993,11475.241032,10321.103072,8265.680152,5754.753607
2048,HLA-A0201,AAAQGQAPL,9,19998.618696,5701.642723,12852.866599,3.006076e+03,5708.723809,7964.493206,4801.535530,2936.617964,13800.948472,9387.764860,4522.826919,4106.470364
2049,HLA-A0201,AADSFATSY,9,19998.618696,28575.905434,24831.331053,9.225714e+04,12821.221997,23734.499920,24568.236374,23071.122753,41074.578924,25536.178860,23586.637703,22361.435147
2050,HLA-A0201,AALGLWLSV,9,2999.162519,376.703799,187.068214,4.064433e+02,32.228609,173.811978,138.485047,213.337331,9.232739,255.139086,166.805578,245.608203
2051,HLA-A0201,AALQGGGPP,9,65012.969034,30902.954325,41304.750199,4.954502e+05,20562.897685,27981.321358,27475.453294,17675.394329,41960.382766,28952.170720,26534.438203,29941.184715
2052,HLA-A0201,AARAALQGG,9,69502.431759,31915.378551,40086.671763,3.981072e+05,20830.615279,35622.807751,25718.333169,23917.732805,31038.723941,25308.593478,25581.803383,22149.000470
2053,HLA-A0201,AARHKHQVM,9,19998.618696,30760.968147,40271.703433,5.382698e+05,44191.411552,22417.930718,27073.986328,19761.560747,38459.798731,27472.976045,26982.928500,20181.976558
2054,HLA-A0201,AAVKAGAAL,9,50582.466200,16788.040181,13396.766874,1.462177e+04,11950.640979,21279.587390,11122.918036,5170.245731,11981.385451,10639.025975,11146.681529,5902.851165
2055,HLA-A0201,AAYARAAAL,9,1640.589773,4375.221052,3564.511334,2.710192e+03,7088.977556,4502.345640,3873.204597,2802.067554,3509.865969,5018.579996,4066.813463,2842.808738
2056,HLA-A0201,ADVRALGGL,9,69502.431759,28379.190284,35892.193465,6.591739e+04,15564.756311,23974.737281,20298.570103,12432.713963,35391.558268,22372.093070,18554.847228,24843.777035


In [29]:
scores_df = collections.defaultdict(list)
predictors = validation_df_with_mhcflurry_results.columns[4:]

for (allele, grouped) in validation_df_with_mhcflurry_results.groupby("allele"):
    scores_df["allele"].append(allele)
    scores_df["test_size"].append(len(grouped.meas))
    for predictor in predictors:
        scores = make_scores(grouped.meas, grouped[predictor])
        for (key, value) in scores.items():
            scores_df["%s_%s" % (predictor, key)].append(value)
    
scores_df = pandas.DataFrame(scores_df)
scores_df.sort("test_size", ascending=False, inplace=True)
scores_df.index = scores_df.allele

scores_df

/home/tim/anaconda2/envs/standard-2.7/lib/python2.7/site-packages/ipykernel/__main__.py:13: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,allele,mhcflurry 0_auc,mhcflurry 0_f1,mhcflurry 0_tau,mhcflurry 1_auc,mhcflurry 1_f1,mhcflurry 1_tau,mhcflurry 2_auc,mhcflurry 2_f1,mhcflurry 2_tau,...,netmhc_auc,netmhc_f1,netmhc_tau,netmhcpan_auc,netmhcpan_f1,netmhcpan_tau,smmpmbec_cpp_auc,smmpmbec_cpp_f1,smmpmbec_cpp_tau,test_size
allele,,,,,,,,,,,,,,,,,,,,,
HLA-A0201,HLA-A0201,0.899123,0.837826,0.554988,0.926853,0.870671,0.623314,0.925713,0.881167,0.622557,...,0.932234,0.884336,0.635498,0.930479,0.880963,0.637338,0.927358,0.885121,0.626224,2126
HLA-A0301,HLA-A0301,0.891561,0.834236,0.540025,0.922007,0.892147,0.595890,0.923343,0.879402,0.602523,...,0.935154,0.900621,0.629236,0.927287,0.885106,0.611240,0.933966,0.897275,0.610891,811


In [37]:
old_scores = pandas.read_csv("../data/validation_scores.csv")
combined_scores = pandas.concat([scores_df, old_scores])

combined_scores["train_size"] = [len(set(all_train_data[a].original_peptides)) for a in combined_scores.allele]

combined_scores.to_csv("../data/validation_scores.csv", index=False)

In [16]:
scores_df.to_csv("../data/validation_scores.csv", index=False)

In [22]:
pandas.DataFrame(models_params_list).to_csv("../data/validation_models.csv", index=False)

In [ ]:
summary_df = scores_df[scores_df.allele.isin(alleles)]
summary_df